Rahmanda Afebrio Yuris Soesatyo - Chapter 14:,Deep Computer Vision Using Convolutional Neural Networks

1. Convolutional and Pooling Layers

Convolutional layer adalah blok utama CNN: neuron tidak lagi terhubung ke seluruh piksel gambar, tetapi hanya ke area lokal kecil yang disebut receptive field. Dengan menggeser receptive field ini di seluruh input, layer pertama mempelajari fitur rendah (tepi, garis), dan layer berikutnya mengkombinasikan fitur ini menjadi pola yang lebih kompleks (sudut, tekstur, objek), membentuk hierarki representasi visual.​

Setiap filter (kernel) adalah matriks bobot kecil yang jika diaplikasikan ke gambar akan menghasilkan feature map yang menonjolkan pola tertentu (misalnya garis vertikal atau horizontal). Banyak filter dalam satu layer menghasilkan banyak feature map, dan parameter sharing (bobot filter yang sama dipakai di semua lokasi) sangat mengurangi jumlah parameter serta membuat model translationally equivariant (pola yang sama dikenali di lokasi berbeda).​

Pooling layer (umumnya MaxPool2D) melakukan subsampling dengan mengambil agregasi (max atau mean) dari patch lokal (misal 2×2) untuk mengurangi resolusi spasial, komputasi, dan sedikit meningkatkan invariansi terhadap translasi kecil. Pooling bisa dilakukan per-channel (umum) maupun depthwise (untuk belajar invariansi terhadap variasi seperti rotasi atau warna).

In [1]:

import tensorflow as tf
from tensorflow import keras

# Simple conv + pooling block
model = keras.models.Sequential([
    keras.layers.Conv2D(
        filters=32, kernel_size=3, strides=1,
        padding="same", activation="relu",
        input_shape=[28, 28, 1]          # grayscale MNIST-like
    ),
    keras.layers.MaxPool2D(pool_size=2),  # halve H and W
    keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2. A Typical CNN for Image Classification

Arsitektur CNN klasik menumpuk beberapa blok Conv2D(+ReLU) lalu MaxPool2D, diakhiri dengan flatten dan beberapa Dense layer untuk klasifikasi. Resolusi spasial berkurang bertahap (karena stride/pooling), sedangkan kedalaman (jumlah feature maps) biasanya meningkat, karena kombinasi fitur rendah menghasilkan banyak kombinasi fitur tinggi.​

Contoh pada Fashion MNIST: pertama Conv besar (64 filter 7×7) diikuti pooling, lalu blok conv berturut-turut dengan jumlah filter bertambah (128, 256) dan pooling, kemudian dua Dense layer dengan dropout dan output softmax. Model semacam ini mencapai akurasi > 92% di Fashion MNIST, jauh lebih baik daripada MLP murni.

In [2]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Conv2D(
        64, 7, activation="relu", padding="same",
        input_shape=[28, 28, 1]
    ),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"),
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"],
)

3. Modern CNN Architectures (LeNet, AlexNet, GoogLeNet, ResNet, Xception, SENet)

Perkembangan arsitektur Convolutional Neural Networks (CNN) modern sangat dipacu oleh kompetisi ImageNet Large Scale Visual Recognition Challenge (ILSVRC), yang mendorong desain jaringan semakin dalam, efisien, dan akurat.

LeNet-5 (1998)
Merupakan CNN klasik yang dikembangkan untuk pengenalan digit MNIST. Arsitekturnya relatif kecil dan dangkal, terdiri dari convolution layer, average pooling, dan beberapa fully connected layer. Meskipun sederhana, LeNet sudah memperkenalkan konsep penting seperti local receptive fields dan weight sharing.

AlexNet (2012)
Menandai terobosan besar dalam deep learning untuk visi komputer. AlexNet jauh lebih dalam dan lebar dibanding LeNet, serta memperkenalkan penggunaan ReLU, dropout, data augmentation, dan local response normalization (LRN). Model ini secara drastis mengungguli metode lain di ImageNet dan memicu kebangkitan deep CNN.

GoogLeNet / Inception (2014)
Memperkenalkan Inception module, yaitu struktur paralel yang mengombinasikan convolution 1×1, 3×3, 5×5, dan max-pooling dalam satu blok. Penggunaan 1×1 convolution sebagai bottleneck membuat model sangat dalam namun tetap efisien secara parameter.

ResNet (2015)
Mengatasi masalah degradasi performa pada jaringan sangat dalam dengan memperkenalkan residual connections berbentuk skip connection (x + F(x)). Konsep residual learning memungkinkan pelatihan jaringan dengan ratusan bahkan ribuan layer secara stabil dan efektif.

Xception (2016)
Merupakan ekstremisasi dari Inception dengan mengganti modulnya menggunakan depthwise separable convolutions, yang memisahkan pemrosesan spasial dan lintas-channel. Pendekatan ini meningkatkan efisiensi komputasi dan sering memberikan akurasi lebih baik pada dataset besar.

SENet (2017)
Menambahkan Squeeze-and-Excitation (SE) blocks ke dalam arsitektur CNN seperti ResNet atau Inception. Blok ini melakukan feature map recalibration dengan memodelkan dependensi antar-channel menggunakan konteks global, sehingga meningkatkan performa secara signifikan.

In [3]:
from tensorflow import keras

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides,
                                padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1,
                                padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides,
                                    padding="same", use_bias=False),
                keras.layers.BatchNormalization(),
            ]

    def call(self, inputs):
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers:
            skip_z = layer(skip_z)
        return self.activation(z + skip_z)

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(
    64, 7, strides=2, padding="same",
    use_bias=False, input_shape=[224, 224, 3]
))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))

prev_filters = 64
for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Dense(10, activation="softmax"))

4. Data Augmentation and Transfer Learning for Image Classification

Data augmentation bertujuan memperbesar dataset training secara artifisial dengan menghasilkan variasi realistis dari gambar asli, seperti:

rotasi kecil,
translasi dan zoom,
flip horizontal,
perubahan brightness atau kontras.
Teknik ini bertindak sebagai regularizer, karena memaksa model mempelajari representasi yang lebih invariant terhadap transformasi yang tidak mengubah label kelas.

Transfer Learning
Untuk tugas visi komputer yang kompleks dan dataset terbatas, pendekatan yang umum adalah transfer learning, yaitu:

Menggunakan CNN besar yang telah dilatih di ImageNet (misalnya ResNet-50, Xception) sebagai feature extractor.
Mengganti classifier di bagian atas dengan head baru (biasanya Global Average Pooling + Dense).
Melatih top layers terlebih dahulu.
Melakukan fine-tuning sebagian atau seluruh base model dengan learning rate kecil.
Strategi ini secara signifikan mempercepat training dan meningkatkan performa.

In [ ]:

from tensorflow import keras
import tensorflow as tf

n_classes = 5  # contoh

base_model = keras.applications.xception.Xception(
    weights="imagenet", include_top=False
)

# Freeze base at first
for layer in base_model.layers:
    layer.trainable = False

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])
    image = keras.applications.xception.preprocess_input(image)
    return image, label

batch_size = 32
train_set = train_set.shuffle(1000).map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)

history = model.fit(train_set, epochs=5, validation_data=valid_set)

# Fine-tune: unfreeze base with smaller learning rate
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

history_fine = model.fit(train_set, epochs=10, validation_data=valid_set)


5. Detection, Localization, and Segmentation

Selain klasifikasi gambar, CNN juga digunakan untuk berbagai tugas visi tingkat lanjut.

Classification + Localization
Model memprediksi kelas objek sekaligus koordinat bounding box (biasanya empat nilai ter-normalisasi). Evaluasi sering menggunakan metrik Intersection over Union (IoU) dan variasi mean Average Precision (mAP).

Object Detection
Bertujuan mendeteksi banyak objek dalam satu gambar. Pendekatan klasik menggunakan sliding window dan CNN, namun metode modern berbasis Fully Convolutional Networks (FCN) seperti YOLO dan SSD mampu memprediksi banyak bounding box dan kelas sekaligus dalam satu forward pass, sehingga jauh lebih efisien.

Semantic Segmentation
Mengklasifikasikan setiap piksel dalam gambar (misalnya jalan, kendaraan, pejalan kaki). Umumnya menggunakan arsitektur encoder–decoder, di mana CNN bertindak sebagai encoder, lalu decoder melakukan upsampling (transposed convolution) dengan bantuan skip connections untuk memulihkan resolusi spasial.

In [5]:
from tensorflow import keras

# Suppose 'cnn' is a conv backbone that outputs feature maps of shape [7, 7, 512]
backbone_output = keras.layers.Input(shape=[7, 7, 512])

# Dense(10) equivalent: Conv2D with 10 filters of size 7x7, valid padding
logits = keras.layers.Conv2D(
    filters=10, kernel_size=7, strides=1, padding="valid"
)(backbone_output)  # output shape: [1, 1, 10]

# If we feed a larger image, backbone_output becomes [H', W', 512],
# and logits becomes [H'-6, W'-6, 10] => grid of predictions (detection-like)
fcn_model = keras.Model(inputs=backbone_output, outputs=logits)
